In [14]:
import os

# install latest pytorch prebuilt with the default prebuilt CUDA version (usually the latest)
!pip install torch==1.5.0 torchvision==0.6.0 cudatoolkit=10.1 -c torch

# install the latest mmcv-full
!pip install mmcv-full==1.3.4

# install mmdetection
!pip install mmdet==2.11.0

# install mmocr
#!git clone https://github.com/open-mmlab/mmocr.git
os.chdir("./mmocr-main")
#os.chdir("/content/drive/MyDrive/AICUP_SceneTextDetection/mmocr-main")

!pip install -r requirements.txt
!pip install -v -e .  # or "python setup.py develop"
!export PYTHONPATH=$(pwd):$PYTHONPATH
    
os.chdir("..")

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'torch'
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/bin
sysconfig: /usr/bin
Value for scheme.data does not match. Please report this to <https://github.com/pypa/pip/is

In [ ]:
!python mmocr-main/tools/train.py mmocr-main/configs/textdet/maskrcnn/public_deform_mask_rcnn_101.py --work-dir mmocr-main/work_dirs/exp --no-validate

fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
2021-06-11 12:20:20,572 - mmocr - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.10 (default, May  3 2021, 02:48:31) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla T4
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.0_bu.TC445_37.28845127_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.8.1+cu101
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.7.0 (Git Hash 7aed236906b1f7a05c0917e5257a1af05e9ff683)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_5

In [ ]:
!python mmocr-main/tools/test.py mmocr-main/configs/textdet/maskrcnn/private_deform_mask_rcnn_101.py mmocr-main/work_dirs/exp/best.pth --out mmocr-main/work_dirs/exp/private_ocr_results.pkl

loading annotations into memory...
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/registry.py", line 51, in build_from_cfg
    return obj_cls(**args)
  File "/content/drive/MyDrive/AICUP_SceneTextDetection/mmocr-main/mmocr/datasets/icdar_dataset.py", line 29, in __init__
    seg_prefix, proposal_file, test_mode, filter_empty_gt)
  File "/usr/local/lib/python3.7/dist-packages/mmdet/datasets/custom.py", line 87, in __init__
    self.data_infos = self.load_annotations(self.ann_file)
  File "/content/drive/MyDrive/AICUP_SceneTextDetection/mmocr-main/mmocr/datasets/icdar_dataset.py", line 41, in load_annotations
    self.coco = COCO(ann_file)
  File "/usr/local/lib/python3.7/dist-packages/pycocotools/coco.py", line 84, in __init__
    with open(annotation_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'PrivateTestDataset/instances_private_test.json'

During handling of the above exception, another exception occurred:

T

In [ ]:
!python mmocr-main/tools/test.py mmocr-main/configs/textdet/maskrcnn/public_deform_mask_rcnn_101.py mmocr-main/work_dirs/exp/best.pth --out mmocr-main/work_dirs/exp/public_ocr_results.pkl

loading annotations into memory...
Done (t=0.51s)
creating index...
index created!
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Use load_from_local loader
[                                                  ] 0/1000, elapsed: 0s, ETA:/usr/local/lib/python3.7/dist-packages/mmdet/models/dense_heads/rpn_head.py:192: UserWarning: In rpn_proposal or test_cfg, nms_thr has been moved to a dict named nms as iou_threshold, max_num has been renamed as max_per_img, name of original arguments and the way to specify iou_threshold of NMS will be deprecated.
  'In rpn_proposal or test_cfg, '
[>>] 1

In [15]:
import numpy as np 
import scipy as sp
import scipy.misc, scipy.ndimage.interpolation
import pandas as pd
import os
import nibabel as nib
import cv2
import matplotlib.pyplot as plt 
from skimage import data, exposure
from glob import glob
import json
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')

In [19]:
import mmcv
import json
with open('PublicTestDataset/instances_public_test.json', 'r') as f:
    json_file = json.load(f)
results = mmcv.load('mmocr-main/work_dirs/exp/public_ocr_results.pkl', 'pkl')

preds = []
for idx, result in enumerate(results):
  for boundary in result['boundary_result']:
    if boundary[-1] > 0.6:
      pred = [idx + 1]
      pred.extend(np.array(boundary)[:-1].astype('int32').flatten())
      pred.append(boundary[-1])
      preds.append(pred)

with open('PrivateTestDataset/instances_private_test.json', 'r') as f:
    json_file = json.load(f)
results = mmcv.load('mmocr-main/work_dirs/exp/private_ocr_results.pkl', 'pkl')

for idx, result in enumerate(results):
  for boundary in result['boundary_result']:
    if boundary[-1] > 0.6:
      pred = [idx + 3001]
      pred.extend(np.array(boundary)[:-1].astype('int32').flatten())
      pred.append(boundary[-1])
      preds.append(pred)


pd.DataFrame(preds).to_csv('submission.csv', index=False, header=False)